# Chapter 12: Tools and Functions - Solutions
**From: Zero to AI Agent**

**Try the exercises in the main notebook first before viewing solutions!**

In [ ]:
!pip install -q -r requirements.txt

from dotenv import load_dotenv
load_dotenv()

print("Setup complete!")

---
## Section 12.1 Solutions: Why Tools Matter

### Exercise 12.1.1 Solution: Tool or No Tool?

Let's analyze each scenario:

1. **Write a haiku about summer**
   - **No tool needed**: LLM can handle this
   - **Why**: Creative writing task, requires language generation
   - **If tool used anyway**: Unnecessary complexity, slower response

2. **Get today's date**
   - **Tool needed**: Yes - Date/time tool
   - **Why**: Current, real-time information
   - **Tool type**: System date tool
   - **If no tool**: Would give wrong or no date

3. **Explain quantum physics**
   - **No tool needed**: LLM has this knowledge
   - **Why**: Established concept in training data
   - **If tool used**: Would just slow down the response unnecessarily

4. **Check if a file exists**
   - **Tool needed**: Yes - File system tool
   - **Why**: Needs to interact with actual file system
   - **Tool type**: File checking tool
   - **If no tool**: Cannot access file system

5. **Translate 'hello' to Spanish**
   - **No tool needed**: LLM knows common translations
   - **Why**: Basic language knowledge in training
   - **If tool used**: Overkill for such a simple translation

6. **Find the latest news about AI**
   - **Tool needed**: Yes - News search tool
   - **Why**: "Latest" indicates current information needed
   - **Tool type**: News API or web search
   - **If no tool**: Would only have information up to training cutoff

7. **Generate a business name**
   - **No tool needed**: LLM excels at creative generation
   - **Why**: Creative task requiring language skills
   - **If tool used**: No appropriate tool exists for creativity

8. **Calculate compound interest**
   - **Tool needed**: Yes - Financial calculator tool
   - **Why**: Precise financial calculations required
   - **Tool type**: Compound interest calculator
   - **If no tool**: Risk of calculation errors, especially with complex formulas

**Key Pattern**: Tools for current data and calculations, LLM for creativity and knowledge!

### Exercise 12.1.2 Solution: Trace the Tool Flow

Here's the complete flow for the conversation:

**Step 1**: User sends message with two requests
- Input: "Search for information about LangChain and calculate 15% of 2000"

**Step 2**: AI parses and recognizes two distinct tasks
- Task A: Search for LangChain information (needs search tool)
- Task B: Calculate 15% of 2000 (needs calculator tool)

**Step 3**: AI decides tool execution order
- Likely executes search first (more complex, takes longer)
- Plans to execute calculator second

**Step 4**: First tool execution - Search
- Tool call: `search_tool("LangChain")`
- Tool returns: Information about LangChain framework, Harrison Chase, etc.

**Step 5**: Second tool execution - Calculator
- Tool call: `calculator_tool("0.15 * 2000")` or `calculator_tool("2000 * 0.15")`
- Tool returns: "300"

**Step 6**: AI processes both results
- Formats search results into coherent paragraph
- Formats calculation result into clear statement

**Step 7**: AI combines results into single response
- Structures response with both answers
- Maintains conversational flow
- Returns complete answer to user

**Key Insights**:
- AI can recognize multiple tasks in one request
- Tools can be executed sequentially
- Results are combined intelligently
- The AI maintains context throughout

### Exercise 12.1.3 Solution: Design a Tool Set for a Bakery

Here are 5 essential tools for a bakery AI assistant:

**Tool 1: order_manager**
- **Purpose**: Create, view, and update customer orders
- **When used**: "Add a birthday cake order for Saturday" or "What orders do we have for tomorrow?"
- **Inputs**: Action (create/read/update), order details (customer, items, date, special requests)
- **Outputs**: Order confirmation or list of orders with details
- **Safety**: Validate dates are future dates, check item availability, require customer contact info

**Tool 2: recipe_calculator**
- **Purpose**: Scale recipes up or down based on needed quantities
- **When used**: "How much flour do I need to make 5 dozen cookies?" or "Scale the bread recipe for 20 loaves"
- **Inputs**: Recipe name, desired quantity, unit (dozens, loaves, etc.)
- **Outputs**: Adjusted ingredient list with precise measurements
- **Safety**: Validate recipe exists, check for reasonable quantities (not 10,000 cookies), warn about oven capacity

**Tool 3: price_calculator**
- **Purpose**: Calculate prices for custom orders including ingredients, labor, and markup
- **When used**: "How much should we charge for a 3-tier wedding cake?" or "What's the cost breakdown for 100 cupcakes?"
- **Inputs**: Item type, quantity, special requirements (decorations, delivery, etc.)
- **Outputs**: Itemized cost breakdown and suggested retail price
- **Safety**: Ensure minimum price thresholds, validate against pricing rules, flag unusual requests

**Tool 4: supplier_contact**
- **Purpose**: Check supplier prices, availability, and place ingredient orders
- **When used**: "Order more vanilla extract" or "Check flour prices from our suppliers"
- **Inputs**: Ingredient name, quantity needed, urgency level
- **Outputs**: Supplier options with prices, availability, and order confirmation
- **Safety**: Require approval for orders over certain amount, verify supplier is approved, check budget limits

**Tool 5: daily_prep_list**
- **Purpose**: Generate preparation schedule based on orders and recipes
- **When used**: "What needs to be started tonight for tomorrow?" or "Create tomorrow's baking schedule"
- **Inputs**: Date, available staff, oven capacity
- **Outputs**: Time-based task list with priorities and assigned stations
- **Safety**: Check for conflicts, ensure critical items are prioritized, validate against operating hours

**Bonus Tool 6: allergy_checker**
- **Purpose**: Verify ingredients against customer allergies and dietary restrictions
- **When used**: "Can we make this gluten-free?" or "Check if the chocolate cake is nut-free"
- **Inputs**: Recipe or item name, dietary restrictions
- **Outputs**: Safe/unsafe determination with specific ingredients of concern
- **Safety**: Always err on side of caution, include disclaimer about cross-contamination

These tools transform the AI from a chatbot into a true bakery operations assistant!

---
## Section 12.2 Solutions: Creating Custom Tools

### Exercise 12.2.1 Solution: Unit Converter Tool

In [ ]:
from langchain_core.tools import Tool

def unit_converter(conversion_request: str) -> str:
    """Convert between common units."""
    try:
        parts = conversion_request.lower().split()
        if len(parts) != 4 or parts[2] != 'to':
            return "Error: Use format '100 meters to feet'"
        
        value = float(parts[0])
        from_unit = parts[1]
        to_unit = parts[3]
        
        # Conversion rates to base units
        conversions = {
            'meters': 1.0, 'feet': 0.3048, 'miles': 1609.34,
            'kilograms': 1.0, 'pounds': 0.453592, 'grams': 0.001,
            'celsius': 'temp', 'fahrenheit': 'temp', 'kelvin': 'temp'
        }
        
        # Temperature conversions
        if from_unit in ['celsius', 'fahrenheit', 'kelvin']:
            if from_unit == 'celsius' and to_unit == 'fahrenheit':
                result = (value * 9/5) + 32
            elif from_unit == 'fahrenheit' and to_unit == 'celsius':
                result = (value - 32) * 5/9
            else:
                return "Temperature conversion not implemented"
            return f"{value} {from_unit} = {result:.2f} {to_unit}"
        
        # Linear conversions
        if from_unit not in conversions or to_unit not in conversions:
            return "Error: Unknown unit"
        
        # Convert through base unit
        base_value = value * conversions[from_unit]
        result = base_value / conversions[to_unit]
        
        return f"{value} {from_unit} = {result:.2f} {to_unit}"
        
    except Exception as e:
        return f"Error: {str(e)}"

# Create the tool
converter_tool = Tool(
    name="UnitConverter",
    func=unit_converter,
    description="Convert units. Format: 'value from_unit to to_unit'"
)

# Test
print(converter_tool.func("100 meters to feet"))
print(converter_tool.func("32 fahrenheit to celsius"))

### Exercise 12.2.2 Solution: Text Analysis Tools

In [ ]:
from langchain_core.tools import Tool
import re

def text_stats(text: str) -> str:
    """Calculate text statistics."""
    if not text:
        return "Error: No text provided"
    
    words = text.split()
    word_count = len(words)
    sentences = re.findall(r'[.!?]+', text)
    sentence_count = len(sentences) if sentences else 1
    avg_word_length = sum(len(word) for word in words) / word_count if word_count > 0 else 0
    
    return f"Words: {word_count}, Sentences: {sentence_count}, Avg word length: {avg_word_length:.1f}"

def extract_keywords(text: str) -> str:
    """Extract top 5 meaningful words."""
    if not text:
        return "Error: No text provided"
    
    stop_words = {'the', 'a', 'an', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'is', 'was', 'are'}
    words = text.lower().split()
    meaningful = [w for w in words if w not in stop_words and len(w) > 2]
    
    word_freq = {}
    for word in meaningful:
        word_freq[word] = word_freq.get(word, 0) + 1
    
    top_words = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)[:5]
    return f"Keywords: {', '.join([w[0] for w in top_words])}"

def summarize(text: str) -> str:
    """Create one-sentence summary."""
    if not text:
        return "Error: No text provided"
    if len(text) < 50:
        return "Text too short to summarize"
    
    sentences = re.split(r'[.!?]+', text)
    first_sentence = sentences[0].strip() if sentences else text[:100]
    return f"Summary: {first_sentence}."

# Create tools
stats_tool = Tool(name="TextStats", func=text_stats, 
                  description="Calculate text statistics")
keywords_tool = Tool(name="ExtractKeywords", func=extract_keywords,
                     description="Extract top keywords from text")
summarize_tool = Tool(name="Summarize", func=summarize,
                      description="Create one-sentence summary")

# Test
sample_text = "Python is a powerful programming language. It is widely used in data science."
print(stats_tool.func(sample_text))
print(keywords_tool.func(sample_text))
print(summarize_tool.func(sample_text))

### Exercise 12.2.3 Solution: File Manager Tool

In [ ]:
from langchain_core.tools import Tool
from datetime import datetime

class SimpleFileSystem:
    """Simulated file system."""
    
    def __init__(self):
        self.files = {}
    
    def parse_command(self, command: str) -> str:
        """Parse and execute file commands."""
        try:
            cmd = command.lower().strip()
            
            if cmd.startswith("create "):
                parts = cmd[7:].split(" with ")
                if len(parts) == 2:
                    filename, content = parts[0].strip(), parts[1].strip()
                    if filename in self.files:
                        return f"Error: {filename} already exists"
                    self.files[filename] = {
                        'content': content,
                        'created': datetime.now().isoformat()
                    }
                    return f"Created {filename}"
                return "Error: Use 'create file.txt with content'"
            
            elif cmd.startswith("read "):
                filename = cmd[5:].strip()
                if filename not in self.files:
                    return f"Error: {filename} not found"
                return f"Content: {self.files[filename]['content']}"
            
            elif cmd == "list files" or cmd == "list":
                if not self.files:
                    return "No files"
                return "Files: " + ", ".join(self.files.keys())
            
            elif cmd.startswith("delete "):
                filename = cmd[7:].strip()
                if filename not in self.files:
                    return f"Error: {filename} not found"
                del self.files[filename]
                return f"Deleted {filename}"
            
            elif cmd.startswith("search "):
                term = cmd[7:].strip()
                matches = []
                for name, data in self.files.items():
                    if term in data['content'].lower():
                        matches.append(name)
                return f"Found in: {', '.join(matches)}" if matches else "No matches"
            
            else:
                return "Commands: create, read, list, delete, search"
                
        except Exception as e:
            return f"Error: {str(e)}"

# Create file system instance
fs = SimpleFileSystem()

# Create tool
file_manager = Tool(
    name="FileManager",
    func=fs.parse_command,
    description="Manage files: 'create file.txt with content', 'read file.txt', 'list files', 'delete file.txt', 'search term'"
)

# Test
print(file_manager.func("create test.txt with Hello World"))
print(file_manager.func("list files"))
print(file_manager.func("read test.txt"))
print(file_manager.func("search Hello"))
print(file_manager.func("delete test.txt"))

---
## Section 12.3 Solutions: Built-in LangChain Tools

### Exercise 12.3.1 Solution: Tool Explorer

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Initialize tools
search_tool = DuckDuckGoSearchRun()
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
)

print("TOOL EXPLORER - COMPARISON STUDY")
print("=" * 60)

# Test DuckDuckGo with different queries
print("\n1. DUCKDUCKGO TESTS")
print("-" * 40)
queries = {
    "news": "AI breakthroughs 2024",
    "facts": "population of Tokyo",
    "tutorial": "Python pandas beginner"
}

for qtype, query in queries.items():
    result = search_tool.run(query)
    print(f"\n{qtype.upper()}: {query}")
    print(f"Result: {result[:150]}...")
    print(f"Best for: {'Current info' if qtype == 'news' else 'Quick facts' if qtype == 'facts' else 'Learning resources'}")

# Test Wikipedia with different topics  
print("\n\n2. WIKIPEDIA TESTS")
print("-" * 40)
topics = {
    "person": "Albert Einstein",
    "place": "Tokyo",
    "concept": "Machine Learning"
}

for ttype, topic in topics.items():
    result = wikipedia_tool.run(topic)
    print(f"\n{ttype.upper()}: {topic}")
    print(f"Result: {result[:150]}...")
    print(f"Best for: {'Biographies' if ttype == 'person' else 'Geography' if ttype == 'place' else 'Definitions'}")

print("\n\nCOMPARISON SUMMARY:")
print("DuckDuckGo: Current events, tutorials, recent info")
print("Wikipedia: Established facts, biographies, concepts")

### Exercise 12.3.2 Solution: Research Workflow

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WriteFileTool
import tempfile
from datetime import datetime

# Initialize tools
search_tool = DuckDuckGoSearchRun()
wikipedia_tool = WikipediaQueryRun(
    api_wrapper=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=1000)
)
workspace = tempfile.mkdtemp(prefix="research_")
write_tool = WriteFileTool(root_dir=workspace)

def research_topic(topic):
    """Research workflow for a topic."""
    print(f"Researching: {topic}")
    print("-" * 40)
    
    # 1. Wikipedia for background
    print("1. Getting background from Wikipedia...")
    wiki_result = wikipedia_tool.run(topic)
    background = wiki_result[:500] if wiki_result else "No Wikipedia entry"
    
    # 2. Search for recent developments
    print("2. Searching for recent news...")
    search_result = search_tool.run(f"{topic} latest news 2024")
    recent = search_result[:500] if search_result else "No recent news"
    
    # 3. Create structured report
    report = f"""# Research Report: {topic}
Date: {datetime.now().strftime('%Y-%m-%d')}

## Background Information
{background}

## Recent Developments
{recent}

## Summary
This report combines encyclopedic knowledge with current developments.
"""
    
    # 4. Save report
    filename = f"{topic.replace(' ', '_').lower()}_report.md"
    write_tool.run({"file_path": filename, "text": report})
    print(f"3. Report saved: {workspace}/{filename}\n")
    
    return report

# Research multiple topics
topics = ["Quantum Computing", "Blockchain Technology"]
for topic in topics:
    research_topic(topic)

print(f"All reports saved in: {workspace}")

### Exercise 12.3.3 Solution: Data Processing Pipeline

In [ ]:
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import WriteFileTool
import tempfile
import json

# Initialize tools
python_tool = PythonREPLTool()
workspace = tempfile.mkdtemp(prefix="data_analysis_")
write_tool = WriteFileTool(root_dir=workspace)

print("DATA PROCESSING PIPELINE")
print("=" * 60)

# 1. Generate sample data
print("\n1. Generating Data...")
generate_code = """
import random
random.seed(42)
data = [round(random.gauss(50, 15), 2) for _ in range(100)]
print(f"Generated {len(data)} data points")
print(f"First 5: {data[:5]}")
"""
python_tool.run(generate_code)

# 2. Calculate statistics
print("\n2. Calculating Statistics...")
stats_code = """
import statistics
data = [45.51, 47.79, 50.59, 29.52, 46.56]  # Sample for demo
mean = statistics.mean(data)
stdev = statistics.stdev(data)
print(f"Mean: {mean:.2f}, Std Dev: {stdev:.2f}")
print(f"Min: {min(data):.2f}, Max: {max(data):.2f}")
"""
python_tool.run(stats_code)

# 3. Create and save report
print("\n3. Creating Report...")
report = """# Data Analysis Report

## Statistics
- Mean: 43.99
- Std Dev: 8.42
- Min: 29.52
- Max: 50.59

## Data Quality
- No missing values
- Normal distribution confirmed
"""

write_tool.run({"file_path": "analysis_report.md", "text": report})
print(f"Report saved: {workspace}/analysis_report.md")

# 4. Test edge cases
print("\n4. Testing Edge Cases...")
edge_code = """
import statistics
# Empty data test
try:
    statistics.mean([])
except statistics.StatisticsError:
    print("Empty data handled correctly")

# Single value test
print(f"Single value: mean={statistics.mean([42])}")

# Invalid types test
try:
    statistics.mean([1, 'two', 3])
except TypeError:
    print("Invalid types caught correctly")
"""
python_tool.run(edge_code)

print(f"\nPipeline complete! Files saved in: {workspace}")

---
## Section 12.4 Solutions: Structured Tool Inputs

### Exercise 12.4.1 Solution: Weather with Units

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from typing import Literal
from dotenv import load_dotenv

load_dotenv()

class WeatherInput(BaseModel):
    city: str = Field(description="City name")
    unit: Literal["celsius", "fahrenheit", "kelvin"] = Field(
        default="celsius",
        description="Temperature unit"
    )

def get_weather_with_units(city: str, unit: str = "celsius") -> str:
    """Get weather with specified temperature unit."""
    # Simulated weather data (always in Celsius)
    weather_data = {
        "London": {"temp_c": 15, "condition": "Cloudy"},
        "New York": {"temp_c": 22, "condition": "Sunny"},
        "Tokyo": {"temp_c": 18, "condition": "Clear"},
    }
    
    if city not in weather_data:
        return f"Weather data not available for {city}"
    
    data = weather_data[city]
    temp_c = data["temp_c"]
    
    # Convert temperature
    if unit == "fahrenheit":
        temp = (temp_c * 9/5) + 32
        symbol = "F"
    elif unit == "kelvin":
        temp = temp_c + 273.15
        symbol = "K"
    else:
        temp = temp_c
        symbol = "C"
    
    return f"Weather in {city}: {temp:.1f}{symbol}, {data['condition']}"

# Create tool with structured input
weather_tool = Tool.from_function(
    func=get_weather_with_units,
    name="GetWeather",
    description="Get weather for a city with temperature unit",
    args_schema=WeatherInput
)

# Test with LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tool = llm.bind_tools([weather_tool])

# Test queries
test_queries = [
    "What's the weather in London?",
    "Weather in New York in Fahrenheit",
    "Tokyo weather in Kelvin"
]

for query in test_queries:
    response = llm_with_tool.invoke([HumanMessage(content=query)])
    if response.tool_calls:
        result = get_weather_with_units(**response.tool_calls[0]['args'])
        print(f"{query} -> {result}")

### Exercise 12.4.2 Solution: Parallel Calculator

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from pydantic import BaseModel, Field
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

class CalculatorInput(BaseModel):
    expression: str = Field(description="Expression to be evaluated in calculator")

def calculator(expression: str) -> str:
    """Perform calculation safely."""
    try:
        # Only allow safe operations
        allowed = set('0123456789+-*/().')
        if all(c in allowed or c.isspace() for c in expression):
            result = eval(expression)
            return f"{result}"
        return "Error: Invalid expression"
    except Exception as e:
        return f"Error: {str(e)}"

calc_tool = Tool.from_function(
    func=calculator,
    name="Calculator",
    args_schema=CalculatorInput,
    description="Calculate math expressions"
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools([calc_tool])

# Test multi-step calculation
queries = [
    "Calculate 15 * 4, 100 / 5, and 78 + 22",
    "What is 25 squared minus 100?",
    "Calculate (10 + 5) * 3"
]

for query in queries:
    print(f"\nQuery: {query}")
    response = llm_with_tools.invoke([HumanMessage(content=query)])
    
    if response.tool_calls:
        print(f"Parallel calculations: {len(response.tool_calls)} operations")
        for i, tool_call in enumerate(response.tool_calls, 1):
            print("tool_call: ", tool_call)
            result = calculator(tool_call['args']['expression'])
            print(f"  {i}. {tool_call['args']['expression']} = {result}")

### Exercise 12.4.3 Solution: Task Manager

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import Tool
from langchain_core.messages import HumanMessage
from pydantic import BaseModel, Field
from typing import Literal, Optional
from datetime import datetime, timedelta
from dotenv import load_dotenv

load_dotenv()

# Simple task storage
tasks = {}
task_id = 0

class AddTaskInput(BaseModel):
    title: str = Field(description="Task title")
    priority: Literal["low", "medium", "high"] = Field(default="medium")
    due_date: Optional[str] = Field(default=None, description="Due date YYYY-MM-DD")

class ListTasksInput(BaseModel):
    filter_by: str = Field(default="all", description="Filter by: all, priority, or date")

class CompleteTaskInput(BaseModel):
    task_id: int = Field(description="Task ID number to mark complete")

def add_task(title: str, priority: str = "medium", due_date: str = None) -> str:
    """Add a new task."""
    global task_id
    task_id += 1
    
    # Parse due date
    if due_date:
        try:
            due = datetime.strptime(due_date, "%Y-%m-%d")
        except:
            return f"Error: Invalid date format. Use YYYY-MM-DD"
    else:
        due = None
    
    tasks[task_id] = {
        "title": title,
        "priority": priority,
        "due_date": due_date,
        "status": "pending"
    }
    return f"Task #{task_id} added: '{title}' [{priority}]"

def list_tasks(filter_by: str = "all") -> str:
    """List tasks filtered by: all, priority, or date."""
    if not tasks:
        return "No tasks"
    
    if filter_by == "priority":
        # Sort by priority
        priority_order = {"high": 0, "medium": 1, "low": 2}
        sorted_tasks = sorted(
            tasks.items(),
            key=lambda x: priority_order.get(x[1]["priority"], 3)
        )
    else:
        sorted_tasks = list(tasks.items())
    
    result = []
    for tid, task in sorted_tasks:
        status = "Done" if task["status"] == "complete" else "Pending"
        result.append(f"[{status}] Task #{tid}: {task['title']} [{task['priority']}]")
    
    return "\n".join(result)

def complete_task(task_id: int) -> str:
    """Mark a task as complete."""
    if task_id not in tasks:
        return f"Error: Task #{task_id} not found"
    tasks[task_id]["status"] = "complete"
    return f"Task #{task_id} marked complete"

# Create tools with proper args_schema
tools = [
    Tool.from_function(
        func=add_task,
        name="AddTask",
        description="Add a new task",
        args_schema=AddTaskInput
    ),
    Tool.from_function(
        func=list_tasks,
        name="ListTasks",
        description="List all tasks",
        args_schema=ListTasksInput
    ),
    Tool.from_function(
        func=complete_task,
        name="CompleteTask",
        description="Mark task complete",
        args_schema=CompleteTaskInput
    )
]

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Test commands
commands = [
    "Add a high priority task to call mom tomorrow",
    "Add task: Review report (low priority)",
    "List all tasks",
    "Mark task 1 as complete"
]

for cmd in commands:
    print(f"\n> {cmd}")
    response = llm_with_tools.invoke([HumanMessage(content=cmd)])
    
    if response.tool_calls:
        tool_call = response.tool_calls[0]
        # Execute the tool
        if tool_call['name'] == 'AddTask':
            result = add_task(**tool_call['args'])
        elif tool_call['name'] == 'ListTasks':
            result = list_tasks(**tool_call['args'])
        elif tool_call['name'] == 'CompleteTask':
            result = complete_task(**tool_call['args'])
        print(result)

---
## Section 12.5 Solutions: Tool Selection and Orchestration

### Exercise 12.5.1 Solution: Distinctive Search Descriptions

In [ ]:
from langchain_core.tools import Tool

# Clear, distinctive names and descriptions for search tools

tools = [
    Tool(
        name="general_web_search",
        func=lambda x: f"Web results for {x}",
        description=(
            "Search the entire web for any topic or information. "
            "Use when: Need broad information, tutorials, or general facts. "
            "Input: Any search query. "
            "Output: Mixed web results from various sources."
        )
    ),
    Tool(
        name="news_search",
        func=lambda x: f"News about {x}",
        description=(
            "Search specifically for recent news articles and current events. "
            "Use when: Need latest updates, breaking news, or recent developments. "
            "Input: News topic or event. "
            "Output: Recent news articles from last 7 days."
        )
    ),
    Tool(
        name="academic_paper_search",
        func=lambda x: f"Academic papers on {x}",
        description=(
            "Search scholarly articles, research papers, and academic journals. "
            "Use when: Need peer-reviewed research, citations, or scientific studies. "
            "Input: Academic topic or research question. "
            "Output: Academic papers with citations and abstracts."
        )
    ),
    Tool(
        name="local_business_search",
        func=lambda x: f"Local businesses: {x}",
        description=(
            "Find local businesses, stores, restaurants, and services. "
            "Use when: Need addresses, hours, reviews for physical locations. "
            "Input: 'business type in location' like 'pizza in NYC'. "
            "Output: Business listings with ratings and contact info."
        )
    ),
    Tool(
        name="social_media_search",
        func=lambda x: f"Social posts about {x}",
        description=(
            "Search social media posts, trends, and discussions. "
            "Use when: Need public opinion, trending topics, or social sentiment. "
            "Input: Topic, hashtag, or person. "
            "Output: Recent social media posts and engagement metrics."
        )
    )
]

# Print the distinctive descriptions
print("DISTINCTIVE SEARCH TOOL DESCRIPTIONS")
print("=" * 60)
for tool in tools:
    print(f"\n{tool.name}:")
    print(f"  {tool.description}")

print("\nEach tool has:")
print("- Clear, specific name")
print("- When to use it")
print("- Input format")
print("- Output description")
print("- No overlap with other tools")

### Exercise 12.5.2 Solution: Loop Prevention

In [ ]:
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

load_dotenv()

# Track calls to detect loops
clarification_count = 0
MAX_CLARIFICATIONS = 1  # Prevent loops

def question_answering(question: str) -> str:
    """Answer questions directly."""
    # Simple Q&A logic
    if "python" in question.lower():
        return "Python is a high-level programming language known for simplicity."
    return f"Here's the answer to '{question}': [detailed answer]"

def ask_clarification(topic: str) -> str:
    """Ask for clarification - potential loop risk!"""
    global clarification_count
    clarification_count += 1
    
    # LOOP PREVENTION: Stop after one clarification
    if clarification_count > MAX_CLARIFICATIONS:
        return "Proceeding with available information."
    
    return f"Could you be more specific about '{topic}'?"

def get_definition(term: str) -> str:
    """Get definition of a term."""
    definitions = {
        "python": "A programming language",
        "api": "Application Programming Interface",
        "llm": "Large Language Model"
    }
    return definitions.get(term.lower(), f"Definition of {term}: [standard definition]")

# Create tools with loop prevention in descriptions
tools = [
    Tool(
        name="answer_question",
        func=question_answering,
        description="Answer questions directly. Always try this first."
    ),
    Tool(
        name="clarify",
        func=ask_clarification,
        description="Ask for clarification ONLY if absolutely needed. Maximum once per conversation."
    ),
    Tool(
        name="define",
        func=get_definition,
        description="Get definition of technical terms"
    )
]

# Test with LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm_with_tools = llm.bind_tools(tools)

# Test queries
test_queries = [
    "What is Python?",
    "Tell me about programming",
    "Define API"
]

print("LOOP PREVENTION DEMONSTRATION")
print("=" * 50)

for query in test_queries:
    clarification_count = 0  # Reset for each query
    print(f"\nQuery: '{query}'")
    
    response = llm_with_tools.invoke([HumanMessage(content=query)])
    if response.tool_calls:
        tool_name = response.tool_calls[0]['name']
        print(f"Tool selected: {tool_name}")
        
        # Execute tool
        tool = next(t for t in tools if t.name == tool_name)
        result = tool.func(query)
        print(f"Result: {result}")
        
        if tool_name == "clarify":
            print(f"Clarification count: {clarification_count}/{MAX_CLARIFICATIONS}")

print("\nLoop Prevention Strategies Applied:")
print("1. Clarification limited to once per query")
print("2. Description indicates 'ONLY if absolutely needed'")
print("3. Counter prevents multiple clarifications")

### Exercise 12.5.3 Solution: Travel Planner Orchestration

In [ ]:
from langchain_core.tools import Tool
from datetime import datetime, timedelta
import random

# Travel planning tools
def get_current_date() -> str:
    """Get today's date."""
    return datetime.now().strftime("%Y-%m-%d")

def calculate_next_month(date: str = None) -> str:
    """Calculate dates for next month."""
    base = datetime.now() if not date else datetime.strptime(date, "%Y-%m-%d")
    next_month = base + timedelta(days=30)
    return f"{next_month.strftime('%Y-%m-%d')} to {(next_month + timedelta(days=2)).strftime('%Y-%m-%d')}"

def check_weather(location: str, dates: str) -> str:
    """Check weather forecast."""
    # Simulated weather
    weather = random.choice(["Sunny", "Partly Cloudy", "Light Rain"])
    temp = random.randint(60, 75)
    return f"Weather in {location} ({dates}): {weather}, {temp}F average"

def search_flights(origin: str, destination: str, dates: str) -> str:
    """Search for flights."""
    # Simulated flight search
    price = random.randint(400, 800)
    return f"Flights {origin} -> {destination} ({dates}): Found 5 options, from ${price}"

def search_hotels(location: str, dates: str, nights: int = 3) -> str:
    """Search for hotels."""
    # Simulated hotel search
    price = random.randint(100, 200)
    return f"Hotels in {location} ({dates}, {nights} nights): 10 options, from ${price}/night"

def find_activities(location: str, interests: str = "general") -> str:
    """Find activities and attractions."""
    activities = {
        "Tokyo": ["Visit Senso-ji Temple", "Explore Shibuya", "Tokyo Tower", "Tsukiji Market"],
        "Paris": ["Eiffel Tower", "Louvre Museum", "Seine River Cruise", "Montmartre"],
        "default": ["City tour", "Local museum", "Food market", "Parks"]
    }
    city_activities = activities.get(location, activities["default"])
    return f"Top activities in {location}: " + ", ".join(city_activities[:3])

def create_itinerary(trip_details: str) -> str:
    """Create final itinerary from all gathered information."""
    return f"""
    TRIP ITINERARY
    ==============
    {trip_details}
    
    Day 1: Arrival and city orientation
    Day 2: Major attractions
    Day 3: Cultural experiences and departure
    
    Total estimated budget: $1,500-2,000 per person
    """

# Create tools with clear orchestration hints
tools = [
    Tool(
        name="get_date",
        func=get_current_date,
        description="Get current date. Use FIRST to establish timeline."
    ),
    Tool(
        name="calculate_dates",
        func=calculate_next_month,
        description="Calculate travel dates for next month. Use AFTER getting current date."
    ),
    Tool(
        name="check_weather",
        func=lambda x: check_weather(*x.split("|")),
        description="Check weather. Input: 'location|dates'. Use AFTER establishing dates."
    ),
    Tool(
        name="search_flights",
        func=lambda x: search_flights(*x.split("|")),
        description="Search flights. Input: 'origin|destination|dates'. Can use PARALLEL with hotels."
    ),
    Tool(
        name="search_hotels",
        func=lambda x: search_hotels(*x.split("|")),
        description="Search hotels. Input: 'location|dates|nights'. Can use PARALLEL with flights."
    ),
    Tool(
        name="find_activities",
        func=lambda x: find_activities(x),
        description="Find activities for a location. Can use ANYTIME after location known."
    ),
    Tool(
        name="create_itinerary",
        func=create_itinerary,
        description="Create final itinerary. Use LAST after gathering all information."
    )
]

# Demonstrate orchestration sequence
print("TRAVEL PLANNING ORCHESTRATION")
print("=" * 60)
print("\nQuery: 'Plan a 3-day trip to Tokyo next month'\n")
print("EXPECTED ORCHESTRATION SEQUENCE:")
print("1. get_date() -> Know current date")
print("2. calculate_dates() -> Determine next month dates")
print("3. PARALLEL:")
print("   - check_weather('Tokyo|dates')")
print("   - search_flights('MyCity|Tokyo|dates')")
print("   - search_hotels('Tokyo|dates|3')")
print("   - find_activities('Tokyo')")
print("4. create_itinerary(all_gathered_info)")
print("\nTools designed for natural orchestration flow!")

# Test individual tools
print("\nTOOL EXECUTION SAMPLES:")
print("-" * 40)
current = get_current_date()
print(f"Current date: {current}")

dates = calculate_next_month()
print(f"Travel dates: {dates}")

weather = check_weather("Tokyo", dates)
print(f"Weather: {weather}")

flights = search_flights("New York", "Tokyo", dates)
print(f"Flights: {flights}")

activities = find_activities("Tokyo")
print(f"Activities: {activities}")

---
## Section 12.6 Solutions: Error Handling

### Exercise 12.6.1 Solution: Robust URL Fetcher

In [ ]:
import time
import re
from typing import Optional

def robust_url_fetcher(url: str) -> str:
    """Fetch webpage content with validation, timeout, and retry."""
    
    # 1. URL Validation
    if not url:
        return "Error: URL cannot be empty"
    
    # Check for valid protocol
    if not url.startswith(('http://', 'https://')):
        return "Error: URL must start with http:// or https://"
    
    # Basic URL pattern check
    url_pattern = re.compile(
        r'^https?://'  # http:// or https://
        r'(?:[A-Za-z0-9.-]+)'  # domain
        r'(?::\d+)?'  # optional port
        r'(?:/[^?]*)?'  # path
    )
    
    if not url_pattern.match(url):
        return "Error: Invalid URL format"
    
    # 2. Fetch with timeout and retries
    MAX_RETRIES = 2
    TIMEOUT = 5
    
    for attempt in range(MAX_RETRIES + 1):
        try:
            # Simulate URL fetching with timeout
            # In production, use: requests.get(url, timeout=TIMEOUT)
            
            # Simulate different scenarios
            import random
            if random.random() < 0.3:  # 30% failure rate
                raise TimeoutError("Request timed out")
            
            # Simulate successful fetch
            return f"Successfully fetched content from {url}: [webpage content here]"
            
        except TimeoutError:
            if attempt < MAX_RETRIES:
                print(f"  Attempt {attempt + 1} timed out, retrying...")
                time.sleep(1)  # Wait before retry
            else:
                return f"Error: Could not fetch {url} - connection timed out after {MAX_RETRIES + 1} attempts"
        
        except Exception as e:
            if attempt < MAX_RETRIES:
                print(f"  Attempt {attempt + 1} failed, retrying...")
                time.sleep(1)
            else:
                return f"Error: Could not fetch {url} - {str(e)}"
    
    return f"Error: Unable to fetch {url}"

# Test the robust fetcher
print("ROBUST URL FETCHER TEST")
print("=" * 50)

test_urls = [
    "https://example.com",          # Valid
    "http://test.org/page",         # Valid
    "example.com",                  # Missing protocol
    "",                              # Empty
    "not-a-url",                     # Invalid format
    "ftp://file.com",               # Wrong protocol
]

for url in test_urls:
    print(f"\nFetching: '{url}'")
    result = robust_url_fetcher(url)
    print(f"Result: {result}")

### Exercise 12.6.2 Solution: Smart Data Processor

In [ ]:
def smart_data_processor(data_string: str, max_rows: int = 1000, max_cols: int = 100) -> str:
    """Process data with multiple fallback parsing strategies."""
    
    if not data_string:
        return "Error: No data provided"
    
    lines = data_string.strip().split('\n')
    
    # Try different separators in order
    separators = [
        (',', 'CSV'),
        ('\t', 'TSV'),
        (' ', 'Space-separated'),
        ('|', 'Pipe-separated')
    ]
    
    for separator, format_name in separators:
        try:
            # Try to parse with current separator
            parsed_data = []
            for line in lines:
                if line.strip():  # Skip empty lines
                    row = line.split(separator)
                    parsed_data.append(row)
            
            if not parsed_data:
                continue
            
            # Validation
            num_rows = len(parsed_data)
            num_cols = len(parsed_data[0]) if parsed_data else 0
            
            # Check if all rows have same number of columns
            consistent = all(len(row) == num_cols for row in parsed_data)
            
            if not consistent:
                continue  # Try next separator
            
            # Validate size limits
            if num_rows > max_rows:
                return f"Error: Too many rows ({num_rows}). Maximum allowed: {max_rows}"
            
            if num_cols > max_cols:
                return f"Error: Too many columns ({num_cols}). Maximum allowed: {max_cols}"
            
            # Success!
            return f"""Successfully parsed as {format_name}:
- Rows: {num_rows}
- Columns: {num_cols}
- First row: {parsed_data[0]}
- Data preview: {parsed_data[:3]}"""
            
        except Exception:
            continue  # Try next separator
    
    # If all parsing attempts failed
    sample = data_string[:200] + "..." if len(data_string) > 200 else data_string
    return f"""Error: Could not parse data.

Tried formats: CSV, TSV, Space-separated, Pipe-separated

Expected format examples:
- CSV: name,age,city
- TSV: name[TAB]age[TAB]city
- Space: name age city

Your data sample: {sample}"""

# Test the processor
print("SMART DATA PROCESSOR TEST")
print("=" * 50)

test_data = [
    "name,age,city\nJohn,30,NYC\nJane,25,LA",  # CSV
    "name\tage\tcity\nJohn\t30\tNYC",          # TSV
    "name age city\nJohn 30 NYC",               # Space-separated
    "invalid;;;data;;;format",                  # Invalid
    "",                                          # Empty
]

for i, data in enumerate(test_data, 1):
    print(f"\nTest {i}:")
    print(f"Input: {data[:50]}...")
    result = smart_data_processor(data)
    print(f"Result: {result[:200]}")

### Exercise 12.6.3 Solution: Resilient API Client

In [ ]:
import time
from datetime import datetime, timedelta
from collections import deque
import json

class ResilientAPIClient:
    """API client with rate limiting, backoff, circuit breaker, and caching."""
    
    def __init__(self):
        # Rate limiting
        self.calls_per_minute = 10
        self.call_times = deque()
        
        # Circuit breaker
        self.consecutive_failures = 0
        self.max_failures = 5
        self.circuit_open = False
        self.circuit_open_until = None
        
        # Cache
        self.cache = {}
        self.cache_duration = 300  # 5 minutes
        
        # Logging
        self.logs = []
    
    def _log(self, message: str, level: str = "INFO"):
        """Add to log."""
        entry = {
            "time": datetime.now().isoformat(),
            "level": level,
            "message": message
        }
        self.logs.append(entry)
        print(f"[{level}] {message}")
    
    def _check_rate_limit(self) -> bool:
        """Check if we're within rate limit."""
        now = datetime.now()
        one_minute_ago = now - timedelta(minutes=1)
        
        # Remove old calls
        while self.call_times and self.call_times[0] < one_minute_ago:
            self.call_times.popleft()
        
        return len(self.call_times) < self.calls_per_minute
    
    def _check_circuit(self) -> bool:
        """Check if circuit breaker allows request."""
        if not self.circuit_open:
            return True
        
        if datetime.now() >= self.circuit_open_until:
            self.circuit_open = False
            self.consecutive_failures = 0
            self._log("Circuit breaker reset")
            return True
        
        return False
    
    def call_api(self, endpoint: str, params: dict = None) -> str:
        """Make API call with all resilience features."""
        
        # 1. Check circuit breaker
        if not self._check_circuit():
            wait_seconds = (self.circuit_open_until - datetime.now()).seconds
            return f"Error: Circuit breaker open. Try again in {wait_seconds} seconds"
        
        # 2. Check cache
        cache_key = f"{endpoint}:{json.dumps(params or {})}"
        if cache_key in self.cache:
            cached_time, cached_result = self.cache[cache_key]
            if datetime.now() - cached_time < timedelta(seconds=self.cache_duration):
                self._log(f"Cache hit for {endpoint}")
                return f"[CACHED] {cached_result}"
        
        # 3. Check rate limit
        if not self._check_rate_limit():
            self._log("Rate limit exceeded", "WARNING")
            return "Error: Rate limit exceeded. Maximum 10 calls per minute"
        
        # 4. Make API call with exponential backoff
        max_retries = 3
        
        for attempt in range(max_retries):
            try:
                # Record call time
                self.call_times.append(datetime.now())
                
                # Simulate API call (30% failure rate)
                import random
                if random.random() < 0.3:
                    raise Exception("API error")
                
                # Success!
                result = f"API Response: {endpoint} with {params}"
                
                # Update cache
                self.cache[cache_key] = (datetime.now(), result)
                
                # Reset failure counter
                self.consecutive_failures = 0
                
                self._log(f"Successful API call to {endpoint}")
                return result
                
            except Exception as e:
                self.consecutive_failures += 1
                
                if attempt < max_retries - 1:
                    # Exponential backoff
                    wait_time = 2 ** attempt
                    self._log(f"Attempt {attempt + 1} failed, waiting {wait_time}s", "WARNING")
                    time.sleep(wait_time)
                else:
                    # Final failure
                    self._log(f"All attempts failed for {endpoint}", "ERROR")
                    
                    # Check circuit breaker
                    if self.consecutive_failures >= self.max_failures:
                        self.circuit_open = True
                        self.circuit_open_until = datetime.now() + timedelta(seconds=30)
                        self._log("Circuit breaker opened for 30 seconds", "ERROR")
                    
                    return f"Error: API call failed after {max_retries} attempts"
    
    def get_stats(self) -> str:
        """Get client statistics."""
        return f"""
API Client Statistics
====================
Rate limit: {len(self.call_times)}/{self.calls_per_minute} calls/min
Circuit: {'OPEN' if self.circuit_open else 'CLOSED'}
Consecutive failures: {self.consecutive_failures}
Cache entries: {len(self.cache)}
Log entries: {len(self.logs)}
"""

# Test the resilient client
print("RESILIENT API CLIENT TEST")
print("=" * 50)

client = ResilientAPIClient()

# Test various scenarios
for i in range(15):  # More than rate limit
    result = client.call_api("test/endpoint", {"id": i})
    print(f"Call {i+1}: {result[:50]}...")
    
    # Show stats every 5 calls
    if i % 5 == 4:
        print(client.get_stats())

---
## Congratulations!

You've completed the solutions for Chapter 12: Tools and Functions!

Return to **chapter_12_tools_functions.ipynb** for more practice.

Next: **Chapter 13: Agent Memory**